# core

> Our basic extension to cosette to build agentic workflows. This is just base class that defines a decorator to make a function into an "Agent"

In [ ]:
#| default_exp core


In [ ]:
#| export
from fastcore.test import *
from typing import Callable, Any, Dict
from functools import wraps
from cosette import Chat

In [ ]:
#| hide
from nbdev.showdoc import *
from datetime import datetime
from pprint import pprint
from dotenv import load_dotenv

## Load keys for testing.
load_dotenv()

True

In [ ]:
#| export
def tool(f: Callable) -> Callable:
    "Marks a function as a tool that can be used by agents. Returns the decorated function with `is_tool=True`"
    f.is_tool = True
    return f

In [ ]:
#| export
def agent(role: str) -> Callable:
    "Creates an agent with a specific role and available tools. Returns a decorator that wraps agent functions."
    def _inner(f: Callable) -> Callable:
        @wraps(f)
        def _wrapper(*args, chat=None, **kwargs) -> Dict[str,Any]:
            chat = chat or Chat(model)
            tools = {n:fn for n,fn in f.__globals__.items() if hasattr(fn,'is_tool')}
            chat.system = f"You are a {role}."
            for n,t in tools.items(): chat.add_tool(n,t)
            return f(*args, chat=chat, **kwargs)
        return _wrapper
    return _inner

In [ ]:
#| test
from fastcore.test import *

@tool
def sample_tool(x: int) -> int:
    "A sample tool that doubles its input"
    return x * 2

@agent("calculator")
def use_tool(x: int, chat=None) -> Dict[str,Any]:
    "An agent that uses the sample tool"
    return {'result': sample_tool(x)}

def test_tool_decorator():
    "Test that tool decorator adds is_tool attribute"
    test_eq(hasattr(sample_tool, 'is_tool'), True)
    test_eq(sample_tool(2), 4)

def test_agent_decorator():
    "Test that agent decorator preserves function name and adds chat capability"
    test_eq(use_tool.__name__, 'use_tool')
    result = use_tool(3)
    test_eq(result['result'], 6)

def test_agent_with_chat():
    "Test that agent works with provided chat instance"
    chat = Chat(model)
    chat.system = "Custom system message"
    result = use_tool(3, chat=chat)
    test_eq(result['result'], 6)
    test_eq(chat.system, "You are a calculator.")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()